In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
#from matplotlib import rcParam
import seaborn as sb
import os
import sys
sys.setrecursionlimit(1000000)
from scipy.stats import spearmanr
import pathlib
import scipy as sp
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache()
from scipy.stats import pearsonr



In [ ]:
from matplotlib import rcParams

In [ ]:
#barcode_path_junk_test = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/junk_test/barcodes_across_sample.pkl')
barcode_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/Final_processed_sequences/barcodes_across_sample.pkl')
#barcode_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/barcodes_across_sample.pkl')
barcodes_across_sample = pd.read_pickle(barcode_path)
directory = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/Final_processed_sequences')
#junk_test_directory = barcode_path_junk_test = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/junk_test')
#load registration files containing volume of each brain area within each sample and which RT primer corresponds to which sample name
lcm_reg_dir = pathlib.PurePath('/nemo/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration')
#_3dareas = '/camp/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration/3D_areas_in_sample.csv'
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)

In [ ]:
#determine cut-off for total counts for barcode across dataset
max_y = 100
rcParams['figure.figsize'] = 8, 5
interpolate_on_x = len(np.flip(np.sort(barcodes_across_sample.sum(axis=1))))-len(np.flip(np.sort(barcodes_across_sample.sum(axis=1)))[np.flip(np.sort(barcodes_across_sample.sum(axis=1)))<max_y])
plt.loglog(np.flip(np.sort(barcodes_across_sample.sum(axis=1))))
plt.xlabel('rank')
plt.ylabel('total barcode counts')
plt.axhline(y =max_y, linestyle='dashed', color = "Black", label = "cut-off", alpha=0.5)
plt.axvline(x =interpolate_on_x, linestyle='dashed', color = "Black", label = "cut-off", alpha=0.5)
plt.title(f'total count per barcode across MAPseq dataset ranked (x={interpolate_on_x})')



In [ ]:
#remove barcodes where total barcode count across dataset is below certain limit (max_y in before)
filtered_barcodes =barcodes_across_sample[barcodes_across_sample.sum(axis=1)>=max_y]

In [ ]:
rcParams['figure.figsize'] = 5, 5
cmap = plt.get_cmap("tab20c")

negs = filtered_barcodes[[1, 2, 3, 4, 5]].melt(var_name='samples', value_name='barcode_counts')
colours_picking = sb.color_palette('tab20c')[0:3]
#plt.pie(negs[negs['barcode_counts']>0]['barcode_counts'])
plt.pie(negs[negs['barcode_counts']>0]['barcode_counts'].value_counts(), labels=['1', '2'], colors=colours_picking)
plt.title(f'barcode umi counts in negative controls \n n = {len(negs[negs.barcode_counts>0])}')


In [ ]:
#define minimum barcode count per sample based on negative controls
filtered_barcodes = filtered_barcodes.replace(1,0)
#filtered_barcodes = filtered_barcodes.replace(2,0)

In [ ]:
rcParams['figure.figsize'] = 10, 5
av_count_look = filtered_barcodes.copy()
av_count_look = av_count_look.replace(0, np.NaN)
plt.bar(av_count_look.columns, av_count_look.mean(), yerr=av_count_look.std())
plt.yscale('log')
plt.axhline(y =2, linestyle='dashed', color = "Black", label = "min barcode count (2)", alpha=0.5)
plt.title('Mean count per barcode per sample')
plt.xlabel('sample')
plt.ylabel('mean umi count')
plt.xticks(np.arange(1, 93, 2), rotation=45, fontsize=8)
plt.legend(loc ='upper left')


In [ ]:
to_plot = filtered_barcodes.melt(var_name='samples', value_name='barcode_counts')
to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 1].index)
to_plot['log_bc_counts']= np.log10(to_plot['barcode_counts'])

rcParams['figure.figsize'] = 18, 5


#to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 2].index)
sb.set_style("whitegrid")
#ax = sb.barplot(x="samples", y="barcode_counts", data=to_plot)
# sb.stripplot(x="samples", y="barcode_counts", data=to_plot, ax=ax, color='black')
sb.violinplot(x="samples", y="log_bc_counts", data=to_plot, inner='point', scale="width", palette="tab20c", width=1)
#sb.boxplot(x="samples", y="log_bc_counts", data=to_plot)
#sb.stripplot(x="samples", y="log_bc_counts", data=to_plot)
#plt.yscale('log')
plt.xlabel('sample')
plt.ylabel('log10 per barcode umi count')
plt.xticks(rotation=45, fontsize=8)
plt.title('UMI count per barcode across samples', fontsize=18)

In [ ]:
#Look at total reads per sample
plt.bar(filtered_barcodes.columns, filtered_barcodes.sum(axis=0))
plt.yscale('log')
plt.title('Total reads per sample')
plt.xlabel('sample')
plt.ylabel('# reads')


In [ ]:
#look at total counts for barcode across dataset
max_y = 20
#interpolate_on_x = len(np.flip(np.sort(barcodes_across_sample.sum(axis=1))))-len(np.flip(np.sort(barcodes_across_sample.sum(axis=1)))[np.flip(np.sort(barcodes_across_sample.sum(axis=1)))<max_y])
#plt.loglog(np.flip(np.sort(filtered_barcodes.sum(axis=0))))
plt.plot(np.flip(np.sort(filtered_barcodes.sum(axis=0))))
plt.xlabel('rank')
plt.ylabel('total barcode counts')
plt.yscale('log')
plt.title(f'total count of barcodes per sample across MAPseq dataset ranked')

In [ ]:
#save a dataframe for barcodes that have been filtered based on total overall expression and min count
filtered_barcodes.to_pickle(directory/'filtered_barcode_matrix_for_binary.pkl')

Removing dud, low quality samples using Allen atlas and qPCR datasets

In [ ]:
#plot relative reads to maximum
#load tables
ROI_path = pathlib.Path('/nemo/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/rois')
ROI_table = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/ROI_vol.pkl')
allen_converted_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/LCM_registration/allenccf/allen_ccf_coord')
download_allen = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/Allen_Connectivity')
reads_path = pathlib.Path("/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences")
tube_to_RT = pathlib.Path("/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/tube_to_RT.csv")
finalpix_expt_a = pd.read_pickle('mouse_connectivity/finalpix_expt_a.pkl')
finalpix_expt_b = pd.read_pickle('mouse_connectivity/finalpix_expt_b.pkl')
finalpix_expt_c = pd.read_pickle('mouse_connectivity/finalpix_expt_c.pkl')
#allen anterograde tracing datasets with more than 75% injection site AUDp
experiment_id_a = 120491896 # AUDp
experiment_id_b =116903230 # AUDp, AUDpo, AUDd, AUDv
experiment_id_c =100149109 # AUDp and AUDd
#injection volumes to normalise to (mm3)
expt_a_inj_vol = 0.097
expt_b_inj_vol = 0.114
expt_c_inj_vol= 0.073
# get projection density for each anterograde tracing expt: values are sum of projecting pixels per voxel.
expt_a, pd_a_info = mcc.get_projection_density(experiment_id_a)
expt_b, pd_b_info = mcc.get_projection_density(experiment_id_b)
expt_c, pd_c_info = mcc.get_projection_density(experiment_id_c)

In [ ]:
#generate a table with total number of projection pixels normalised by injection volume for each expt.
all_projection_allen = pd.DataFrame().assign(Tube=finalpix_expt_a['tube'], ExptA = finalpix_expt_a['projection_density_summed']/expt_a_inj_vol, ExptB=finalpix_expt_b['projection_density_summed']/expt_b_inj_vol, ExptC=finalpix_expt_c['projection_density_summed']/expt_c_inj_vol)
#all_projection_allen = all_projection_allen.set_index('Tube')
projection_comp = pd.DataFrame(all_projection_allen.mean(axis=1), columns=['Allen'])
#projection_comp['std_allen'] = all_projection_allen.std(axis=1)
projection_comp['Tube'] = all_projection_allen['Tube']
RT_convert = pd.read_csv(tube_to_RT)
projection_comp.Tube = projection_comp.Tube.astype(str).astype(int)
projection_comp = pd.merge(projection_comp, RT_convert, on ='Tube', how ='inner')
projection_comp['MAPseq_counts'] = projection_comp.apply(lambda x: filtered_barcodes[x['RT primer']].sum(), axis=1)

In [ ]:
#drop sample 5 that has no registration data
projection_comp = projection_comp.drop(projection_comp[projection_comp['RT primer']==5].index)
filtered_barcodes = filtered_barcodes.drop(5, axis=1)

In [ ]:
projection_comp['log_MAPseq_counts']=np.log((projection_comp.MAPseq_counts)+1)
projection_comp['log_Allen']=np.log(projection_comp.Allen)
#projection_comp.replace([np.inf, -np.inf], np.nan, inplace=True)
#projection_comp = projection_comp.dropna()

In [ ]:
rcParams['figure.figsize'] = 8, 5
corr, _ = pearsonr(projection_comp['log_Allen'], y=projection_comp['log_MAPseq_counts'])
print(f'Pearsons correlation: {corr}')
p = sb.regplot(data = projection_comp, x='log_Allen', y='log_MAPseq_counts')
#sb.scatterplot(x=projection_comp[projection_comp['fold_dif']<threshold]['log_Allen'], y=projection_comp[projection_comp['fold_dif']<threshold]['log_MAPseq_counts'], hue=projection_comp[projection_comp['fold_dif']<threshold]['fold_dif'])
#sb.scatterplot(x=projection_comp[projection_comp['y_dif']>offset_thresh]['log_Allen'], y=projection_comp[projection_comp['y_dif']>offset_thresh]['log_MAPseq_counts'], hue=projection_comp[projection_comp['y_dif']>offset_thresh]['y_dif'])
#calculate slope and intercept of regression equation
slope, intercept, r, p, sterr = sp.stats.linregress(x=p.get_lines()[0].get_xdata(),
                                                       y=p.get_lines()[0].get_ydata())


In [ ]:
sp.stats.linregress(x=projection_comp['log_Allen'], y=projection_comp['log_MAPseq_counts'])

In [ ]:
#calculate residuals from regression plot and plot histogram
#plt.rcdefaults()
rcParams['figure.figsize'] = 6, 4
projection_comp['y_dif'] = (slope*projection_comp['log_Allen']+intercept)-projection_comp['log_MAPseq_counts']
plt.hist(projection_comp.y_dif, bins=8)
plt.title('Distribution of residuals')
plt.xlabel('y difference')
plt.ylabel('frequency')

In [ ]:
plt.scatter(projection_comp['log_Allen'], projection_comp['y_dif'])
plt.ylabel('residuals')
plt.xlabel('log_Allen')
rcParams['figure.figsize'] = 4, 4

In [ ]:
projection_comp['y_bounds'] = (projection_comp['log_Allen']*slope)+intercept-(projection_comp['y_dif'].std() *2)
projection_comp['upper_y_bounds'] = (projection_comp['log_Allen']*slope)+intercept+(projection_comp['y_dif'].std() *2)
out_bounds_predictions = projection_comp['y_bounds'] >= projection_comp['log_MAPseq_counts']

In [ ]:
upper_bounds_predictions = projection_comp['upper_y_bounds'] <= projection_comp['log_MAPseq_counts']
projection_comp[upper_bounds_predictions]

In [ ]:
plt.scatter(
    projection_comp[~out_bounds_predictions]['log_Allen'],
    projection_comp[~out_bounds_predictions]['log_MAPseq_counts'],
    color="C0",
    alpha=0.5
)
plt.scatter(
    projection_comp[out_bounds_predictions]['log_Allen'],
    projection_comp[out_bounds_predictions]['log_MAPseq_counts'],
    color="grey",
    marker="+",
    label='outside cut-off'
)
plt.axline((0, intercept), slope=slope, color='C0')
plt.axline((0, intercept-(projection_comp['y_dif'].std() *2)), slope=slope, color='grey', label='+ 2 stdev')
plt.legend()
plt.xlabel('log Allen (AU)')
plt.ylabel('log MAPseq counts')



In [ ]:
#now remove samples that are more than 2 stdev outside residuals
filtered_barcodes_QC = filtered_barcodes.drop(list(projection_comp[out_bounds_predictions]['RT primer']), axis=1)

In [ ]:
bl =filtered_barcodes_QC.sum(axis=0)
#bl = bl[bl>0]

In [ ]:
rcParams['figure.figsize'] = 5, 5
plt.hist(bl[bl<1000], bins=30)
#plt.yscale('log')
plt.axvline(30, color='Black', linestyle='dashed',  alpha=0.5, label='30 barcode cut-off')
plt.xlabel('number of different neuron barcodes')
plt.ylabel('number of samples')
plt.title('Frequency distribution of # individual barcodes per sample, max visualised to 1000')
plt.legend()

In [ ]:
filtered_barcodes_QC = filtered_barcodes_QC.drop(filtered_barcodes_QC.columns[filtered_barcodes_QC.astype(bool).sum()<30], axis=1)




In [ ]:
to_plot = filtered_barcodes_QC.melt(var_name='samples', value_name='barcode_counts')
to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 1].index)
to_plot['log_bc_counts']= np.log10(to_plot['barcode_counts'])

rcParams['figure.figsize'] = 18, 5


#to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 2].index)
sb.set_style("whitegrid")
#ax = sb.barplot(x="samples", y="barcode_counts", data=to_plot)
# sb.stripplot(x="samples", y="barcode_counts", data=to_plot, ax=ax, color='black')
sb.violinplot(x="samples", y="log_bc_counts", data=to_plot, inner='point', scale="width", palette="tab20c", width=1)
#sb.boxplot(x="samples", y="log_bc_counts", data=to_plot)
#sb.stripplot(x="samples", y="log_bc_counts", data=to_plot)
#plt.yscale('log')
plt.xlabel('sample')
plt.ylabel('log10 per barcode umi count')
plt.xticks(rotation=45, fontsize=8)
plt.title('UMI count per barcode across samples', fontsize=18)

In [ ]:
to_plot = filtered_barcodes_QC.melt(var_name='samples', value_name='barcode_counts')
rcParams['figure.figsize'] = 20,10
to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 2].index)
sb.set_style("whitegrid")
ax = sb.barplot(x="samples", y="barcode_counts", data=to_plot)
sb.stripplot(x="samples", y="barcode_counts", data=to_plot, ax=ax, color='black')
#sb.stripplot(x="samples", y="barcode_counts", data=to_plot)
plt.yscale('log')
plt.xlabel('sample')
plt.ylabel('per barcode umi count')
plt.xticks(rotation=45, fontsize=8)
plt.title('UMI count per barcode across samples after removing putative dud samples', fontsize=18)

Normalise samples based on spike-in levels

In [ ]:
#spike-in normalisation, generate table of spike counts per sample
spike_counts = pd.DataFrame(columns=["sample", "spike_count"])
for sample in os.listdir(directory):
    if sample.startswith("spike_counts"):
        sample_name = sample.split("spike_counts_", 1)
        sample_name = sample_name[1][: -len(".csv")]
        sample_num = float(sample_name[2 :])
        sample_reading = pd.read_csv(directory/sample)
        sample_reading["counts"] = sample_reading["counts"].astype("int")
        sum_counts = sample_reading["counts"].sum()
        new_row = pd.DataFrame(
            {"sample": sample_num, "spike_count": sum_counts}, index=[0]
        )
        spike_counts = pd.concat([spike_counts, new_row])


In [ ]:
#plot the distribtution of spike-in reads and decide cut-off threshold
rcParams['figure.figsize'] = 5, 5
plt.hist(spike_counts['spike_count'], bins =20, color='purple')
plt.xlabel('total spike umi count')
plt.ylabel('frequency')
plt.title('distribution of spike counts across samples')
plt.axvline(x = 2000, color = 'black', linestyle='dashed', label = 'cut-off', alpha =0.5)

In [ ]:
#remove spikes that are below certain threshold, then normalise total counts in each sample by relative spike-count

spike_cutoff = 2000
spike_thresh = list(spike_counts[spike_counts['spike_count']<spike_cutoff]['sample'])
filtered_barcodes_QC = filtered_barcodes_QC.drop((spike_thresh), axis=1)

In [ ]:
spikes_thresholded = spike_counts[spike_counts['spike_count']>spike_cutoff]
lowest = min(spikes_thresholded["spike_count"])
spikes_thresholded["normalisation_factor"] = spikes_thresholded["spike_count"] / lowest

In [ ]:
filtered_barcodes_spike_normalised = filtered_barcodes_QC.copy()
for i, row in spikes_thresholded.iterrows():
    if row['sample'] in filtered_barcodes_spike_normalised.columns:
        filtered_barcodes_spike_normalised[row['sample']] = filtered_barcodes_spike_normalised[row['sample']]/row['normalisation_factor']

In [ ]:
#set min to 1
filtered_barcodes_spike_normalised = filtered_barcodes_spike_normalised/filtered_barcodes_spike_normalised[filtered_barcodes_spike_normalised>0].min().min()


In [ ]:
to_plot = filtered_barcodes_spike_normalised.melt(var_name='samples', value_name='barcode_counts')
to_plot = to_plot.drop(to_plot[to_plot.barcode_counts ==0].index)
to_plot['log_bc_counts']= np.log10(to_plot['barcode_counts'])

rcParams['figure.figsize'] = 18, 5


#to_plot = to_plot.drop(to_plot[to_plot.barcode_counts < 2].index)
sb.set_style("whitegrid")
#ax = sb.barplot(x="samples", y="barcode_counts", data=to_plot)
# sb.stripplot(x="samples", y="barcode_counts", data=to_plot, ax=ax, color='black')
sb.violinplot(x="samples", y="log_bc_counts", data=to_plot, inner='point', scale="width", palette="tab20c", width=1)
#sb.boxplot(x="samples", y="log_bc_counts", data=to_plot)
#sb.stripplot(x="samples", y="log_bc_counts", data=to_plot)
#plt.yscale('log')
plt.xlabel('sample')
plt.ylabel('log10 per barcode umi count')
plt.xticks(rotation=45, fontsize=8)
plt.title('UMI count per barcode across samples after spike normalisation', fontsize=18)

In [ ]:
to_plot = filtered_barcodes_spike_normalised.melt(var_name='samples', value_name='barcode_counts')
rcParams['figure.figsize'] = 20,10
post_spike = to_plot[to_plot['barcode_counts']>0]
sb.set_style("whitegrid")
ax = sb.barplot(x="samples", y="barcode_counts", data=post_spike)
sb.stripplot(x="samples", y="barcode_counts", data=post_spike, color='black', ax=ax)
#sb.stripplot(x="samples", y="barcode_counts", data=to_plot)
plt.yscale('log')
plt.xlabel('sample')
plt.ylabel('per barcode umi count')
plt.xticks(rotation=45, fontsize=8)

plt.title('UMI count per barcode across samples after spike normalisation', fontsize=18)

In [ ]:
spikes_thresholded['normalisation_factor'].max()




In [ ]:
bl =filtered_barcodes_spike_normalised.astype(bool).sum(axis=0)
for i in bl:
    print(i)

In [ ]:
#remove any barcodes found only in one sample
filtered_barcodes_spike_normalised = filtered_barcodes_spike_normalised[filtered_barcodes_spike_normalised.astype(bool).sum(axis=1)>1]

#save the qc'd and normalised barcode matrix

filtered_barcodes_spike_normalised.to_pickle(directory/'barcode_matrix_normalised.pkl')

In [ ]:
filtered_barcodes_spike_normalised.to_pickle(directory/'barcode_matrix_normalised.pkl')

In [ ]:
filtered_barcodes_spike_normalised

In [ ]:
#the samples that contain zero spike counts may be an artifact of UMI correction, since we know that there are many MAPseq reads. 
#therefore, let's see if we can correct the artifact and table
BC5 = pd.read_csv('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/corrected_BC57.csv',
        header=0, 
        usecols=["full_read", "neuron_sequence", "corrected_sequences_neuron", "umi_sequence", "corrected_sequences_umi"])
#BC65 = pd.read_csv(directory/'neuron_counts_BC65.csv')

In [ ]:
BC57[BC57['full_read']=='CCTTACCCCCTAAGTGTTGTTTTTTCCTCTTCTATCTTATCACTATACCCGCTTGCGCCTTA']

In [ ]:
#filter ones that contain the spike-in sequence in uncorrected
BC57_spike = BC5[BC5['neuron_sequence'].str.contains("^.{24}ATCAGTCA") == True]

In [ ]:
BC57_spike

In [ ]:
#drop samples that contain spike count less than 1500, as RT likely failed for these samples
min_spike = 1500
spike_thresholded = spike_counts[spike_counts['spike_count'] >= min_spike]
areas_dropped= areas[areas['RT_primer'].isin(spike_thresholded['sample']) == False].RT_primer
areas= areas[areas['RT_primer'].isin(spike_thresholded['sample']) == True]
barcodes_across_sample = barcodes_across_sample.drop(columns=np.array(areas_dropped))    

In [ ]:
#also drop sample 5 that doesn't have reg info
areas = areas.drop([4])
barcodes_across_sample =barcodes_across_sample.drop(columns=[5])

In [ ]:
#now remove any barcodes with a count less than 1, then remove barcodes that don't have a count anywhere.\
barcodes_across_sample = barcodes_across_sample.replace(1,0)
barcodes_across_sample.fillna(0,inplace=True)
barcodes_across_sample = barcodes_across_sample.loc[~(barcodes_across_sample==0).all(axis=1)]


In [ ]:
spike_counts[spike_counts['spike_count'] < min_spike]

In [ ]:
bla = barcodes_across_sample[[1, 2, 3, 4]]
bla = bla.loc[~(bla==0).all(axis=1)]
bla

In [ ]:
#normalise counts by spike-in counts
lowest = min(spike_thresholded["spike_count"])
spike_thresholded["normalisation_factor"] = spike_thresholded["spike_count"] / lowest
#spike_thresholded= spike_thresholded.sort_values("sample", inplace=True)
spike_thresholded =spike_thresholded.set_index('sample')
spike_thresholded.sort_index(inplace=True)
norm = spike_thresholded['normalisation_factor'].T
barcodes_across_sample_changed = barcodes_across_sample.div(norm, axis='columns')

barcodes_across_sample_changed.fillna(0,inplace=True)
#plt heatmap of barcode matrix after spike normalisation
fig, ax = plt.subplots(figsize=(60, 10))
sb.heatmap(barcodes_across_sample, cmap='Blues', norm=LogNorm())
plt.show()

In [ ]:
#label sample 29 as caudal striatum 
CStr = areas[areas['RT_primer']==29]
CStr_val = CStr.sum(axis=1)-(CStr['RT_primer']+CStr['sample'])
to_add = pd.DataFrame(columns=areas.columns)
to_add['Cstr'] = CStr_val
to_add =to_add.fillna(0)
areas['Cstr'] = 0
areas.update(to_add)

In [ ]:
areas['Cstr'] = 0
areas.update(to_add)

In [ ]:
areas[areas['RT_primer']==57]

In [ ]:
group_areas = {
    'tectum': ['SCdg', 'SCdw', 'SCig', 'SCiw', 'SCop', 'SCsg', 'SCzo', 'ICc', 'ICd', 'ICe', 'NB'],
    'thalamus': ['PoT', 'TH', 'MGm', 'MGv', 'MGd', 'LGd-co', 'LP', 'POL', 'PO', 'LD', 'VPL', 'PIL', 'Eth'],
    'SS': ['SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-n', 'SSp-tr', 'SSp-ul', 'SSp-un', 'SSs'],
    'M': ['MOs', 'MOp'],
    'contra': areas.filter(like="Contra").columns,
    #'AudC': ['Contra-AUDd', 'Contra-AUDp', 'Contra-AUDv'],
    #'VisC': ['Contra-VISa', 'Contra-VISam'],
    'VisIP': ['VISa', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISpor', 'VISrl', 'VISli', 'VISpl'],
    'RStr': ['CP', 'STR', 'ACB'],
    'pons': ['SOCm', 'SOCl', 'POR', 'PRNr', 'PRNc', 'TRN', 'P', 'P-mot', 'PG', 'NLL']
}

In [ ]:
for group, columns in group_areas.items():
    areas[group] = areas.filter(items=columns).sum(axis=1)
    areas = areas.drop(columns, axis=1)

In [ ]:
areas_only = areas.drop(['sample', 'RT_primer', 'ar', 'bic', 'bsc', 'ccb', 'ccb', 'ccg', 'cing', 'cpd', 'csc', 'cst', 'ec', 'fa', 'fi',
    'fiber tracts', 'fp', 'll', 'mcp', 'ml', 'onl', 'or', 'py', 'root', 'sctv', 'scwm', 'tb', 'CTXsp', 'act', 'alv', 'amc', 'cic', 'VL', 'MRN'], axis=1)

In [ ]:
areas_only = areas_only.loc[:, np.sum(areas_only, axis=0)>0]
areas_matrix = areas_only.to_numpy()
areas_matrix /= np.sum(areas_matrix, axis=0)

In [ ]:
areas_only.iloc[64]['contra']

In [ ]:

barcodes_across_sample.fillna(0,inplace=True)
barcodes_matrix = barcodes_across_sample.to_numpy()
barcodes_matrix[np.isnan(barcodes_matrix)] = 0
total_projection_strength = np.sum(barcodes_matrix, axis=1)
barcodes_matrix /= total_projection_strength[:, np.newaxis]

barcodes_matrix = barcodes_matrix[total_projection_strength>0, :]

In [ ]:
pd.DataFrame(barcodes_matrix)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso

mdl = LinearRegression(fit_intercept=False, positive=True)
mdl.fit(areas_matrix, barcodes_matrix.T)

In [ ]:
#raw, not spike normalised, with higher cutoff
plt.figure(figsize=(20,70))
df = pd.DataFrame(mdl.coef_[:15000,:], columns=areas_only.columns)
sb.clustermap(df.T, vmax=0.1, dendrogram_ratio=[0.1, 0.1], yticklabels=True)

In [ ]:
areas_comp = ['MOB', 'M', 'Cstr', 'RStr', 'tectum', 'thalamus', 'contra', 'VisIP', 'SS']
areas_grouped = df[areas_comp]

In [ ]:
areas_grouped = areas_grouped.loc[~(areas_grouped==0).all(axis=1)]

In [ ]:
#raw, not spike normalised, with higher cutoff
plt.figure(figsize=(10,50))

sb.clustermap(areas_grouped.T, cmap="Blues", dendrogram_ratio=(.1, .2), vmax=0.05, cbar_pos=(-0.2, .5, .1, .4), yticklabels=True)

In [ ]:
#create a dataframe of the fractions of each brain area contained within each sample
areas_matrix = areas_only.to_numpy()
total_frac = np.sum(areas_matrix, axis=1)
frac_matrix = areas_matrix/total_frac[:, np.newaxis]
areasFrac = pd.DataFrame(frac_matrix, columns=areas_only.columns)

In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(barcodes_across_sample), (len(areas_only.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only.columns, index=barcodes_across_sample.index)

In [ ]:
for i, row in barcodes_across_sample.iterrows():
    bc_matrix1 =pd.DataFrame(columns=areas_only.columns)
    for samplename in barcodes_across_sample.columns:
        counts = row.to_numpy()
        frac_counts =frac_matrix* counts[:, np.newaxis]
        sample_counts =pd.DataFrame(frac_counts, columns = areas_only.columns)
    for region in sample_counts.columns:
        bc_matrix.at[i, region] = sample_counts[region].sum()/areas_only[region].sum()
#bc_matrix.to_pickle(lcm_reg_dir/'bc_matrix_lcm_2.pkl')

In [ ]:
areas_comp = ['MOB', 'M', 'Cstr', 'RStr', 'tectum', 'thalamus', 'contra', 'VisIP', 'SS']
areas_grouped = bc_matrix[areas_comp]

In [ ]:
areas_grouped = areas_grouped.loc[~(areas_grouped==0).all(axis=1)]

In [ ]:
# perform hierarchial clustering of all barcodes across samples
sb.clustermap(areas_grouped, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Blues", figsize=(10, 10))

In [ ]:
sample_counts = pd.DataFrame(frac_counts, columns = areas_only.columns)
for region in sample_counts.columns:
    bc_matrix.at[i, region] = sample_counts[region].sum()/areas_only[region].sum()

In [ ]:
for region in sample_counts.columns:
    bc_matrix.at[i, region] = sample_counts[region].sum()/areas_only[region].sum()

In [ ]:
sample_counts.sum()

In [ ]:
sample_counts['thalamus'].sum()/areas_only[region].sum()

In [ ]:
areas_only[region].sum()

In [ ]:
barcodes_across_sample

In [ ]:
areas_only.iloc[0].sum()

In [ ]:
for i in areas_only[region]:
    print(i)

In [ ]:
row =barcodes_across_sample.iloc[2].to_numpy()

bl =frac_matrix* row[:, np.newaxis]
f =pd.DataFrame(bl, columns = areas_only.columns)

In [ ]:
f

In [ ]:
f.iloc[32]

In [ ]:
ind = -1
for samplename in barcodes_across_sample.columns:
    ind = +1
    bl = row.iloc[ind]*areasFrac
    print(samplename)

In [ ]:
areasFrac.shape

In [ ]:
barcodes_across_sample.loc[2]

In [ ]:
for i in barcodes_across_sample.loc[2]:
    print (i)

In [ ]:
bl =row*areasFrac


In [ ]:
areasFrac

In [ ]:
#for each barcode, create a matrix of BC count for regions in a sample based on amount of each region in LCM (makes assumption of equal BC distribution)
bc_matrix = np.zeros(shape=((len(newdf), (len(areas_only.columns)))))
bc_matrix = pd.DataFrame(data= bc_matrix, columns=areas_only.columns, index=newdf.index)
for i, row in newdf.iterrows():
    bc_matrix1 =pd.DataFrame(columns=areas_only.columns)
    for samplename in newdf.columns:
        ind = areas.index[areas['RT_primer']==samplename].tolist()
        fractionC = areasFrac.iloc[ind[0]]*row.loc[samplename]
        bc_matrix1 = bc_matrix1.append(fractionC)
    for region in bc_matrix1.columns:
        bc_matrix.at[i, region] = bc_matrix1[region].sum()/areas_only[region].sum()
#bc_matrix.to_pickle(lcm_reg_dir/'bc_matrix_lcm_2.pkl')